# RAG — QA sobre documentos que se basa en baseline local

1) Ingesta de documentos y construcción de índice (TF-IDF o embeddings)
Para que corra 100% local sin descargar modelos, uso TF-IDF como baseline.

In [ ]:
# configuración de carpeta raíz al path
import sys, os
sys.path.append(os.path.abspath(".."))  

In [2]:
import os, glob, re, json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

DOC_DIR = "../assets"
docs = []
paths = []
for p in glob.glob(os.path.join(DOC_DIR, "*.txt")):
    with open(p, "r", encoding="utf-8", errors="ignore") as f:
        docs.append(f.read())
        paths.append(p)

if not docs:
    docs = ["Este es un documento de ejemplo sobre deep learning y transformers.",
            "Otro documento acerca de RAG: recuperación y generación con contexto.",
            "Notas de clase sobre CNN y GAN para visión por computadora."]
    paths = ["demo1.txt","demo2.txt","demo3.txt"]

vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(docs)

def retrieve(query, k=3):
    qv = vectorizer.transform([query])
    sims = cosine_similarity(qv, X).ravel()
    idx = np.argsort(-sims)[:k]
    return [(paths[i], sims[i], docs[i][:500]) for i in idx]

def answer(query):
    ctx = retrieve(query, k=3)
    # Generador simple: concatenar contexto más relevante
    context_text = "\n\n".join([c[2] for c in ctx])
    response = f"Pregunta: {query}\n\nContexto:\n{context_text}\n\nRespuesta (heurística): Basado en el contexto, la respuesta se relaciona con los temas recuperados arriba."
    return ctx, response

ctx, resp = answer("¿Qué es un transformer y para qué sirve un RAG?")
resp


'Pregunta: ¿Qué es un transformer y para qué sirve un RAG?\n\nContexto:\nEste es un documento de ejemplo sobre deep learning y transformers.\n\nOtro documento acerca de RAG: recuperación y generación con contexto.\n\nNotas de clase sobre CNN y GAN para visión por computadora.\n\nRespuesta (heurística): Basado en el contexto, la respuesta se relaciona con los temas recuperados arriba.'

In [3]:
from src.utils import save_rag_example
import os
os.makedirs("../results/rag", exist_ok=True)
save_rag_example("../results/rag/examples.jsonl", "¿Qué es un transformer y para qué sirve un RAG?", ctx, resp, correct=None)
print("Guardado en results/rag/examples.jsonl")


Guardado en results/rag/examples.jsonl
